# Capstone Project - The Battle of Neighborhoods #
### _Kitchener Vs. Waterloo, ON_ ###
##### Can a New Sushi Restauraunt be Sucessful in Either Kitchener or Waterloo Neighbourhoods? #####

<img src="https://i.pinimg.com/originals/65/50/cb/6550cb609b6ad9ca31a7795d2b0d786c.jpg" width="400" height="400"/>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for stakeholders to open a new sushi restaurant in either **Kitchener** or **Waterloo**, ON. 

Sushi is a popular choice in the area of Kitchener and Waterloo. This is partly due to the Universities in town and the booming tech scene that supply patrons of these restauraunts. Sushi in the area of Kitchener and Waterloo can be a quick, easy meal and it can also be a tidy meal because it offers many delicious bite size varieties. (No one likes to go back to the office with burger sauce, or salsa on their shirt for example). It's a convenient option that many people enjoy and can be a fun new experiecne for many patrons. It can also provide a very intimate setting throught the idea of food sharing so it is also a versitile setting that can range from romantic date, to family outing, to business meeting.

With so many options of attracting different kinds of patrons with different needs and due to it's popularity in the Kitchener/ Waterloo area it seems natural that stakeholdrs could be interested in investing in a new sushi restauraunt in the area. However, is this type of restauraunt already too puplular in the area? Is there any space left in these neighbourhoods where a new restauraunt could be sucessful? 

To answer these questions we will need to find out if these areas are already too saturated to support another sushi restauraunt. Since there is assumed to be a lot of sushi resatauraunts in the Kitchener/Waterloo area we will try to detect **locations that are not already crowded with sushi restaurants**. We are interested in **areas with little or no sushi restaurants in these neighbourhoods**. We would also prefre locations **as close to the Universities as possible**, assuming that the first two conditions are met. 

We will use data analysis to generate a few of the most promising areas of Kitchener/Waterloo neighbourhoods for a new sushi restauraunt based on this criteria so that a final location can be chosen by stakeholders or so that we can determine that this type of restauraunt is not likely to be sucessful in these areas and perhaps determine that the stakeholders should continue their search in another nearby area such as Cambridge or Elmira,ON. Or, perhaps the data will show that they should look at choosing a different type of restauraunt altogether if they are set on entering the market in these neighbourhoods.  

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to sushi restaurants in the neighborhood, if any
* distance of neighborhood from any Universities (or if no Universities available in the area then Colleges)

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Kitchener/Waterloo center will be obtained using **Google Maps API geocoding** of their respective Universities/Colleges

### Neighborhood Candidates

First we will install our required modules and import any necessary libraries to be able to execute our data requirements.

In [1]:
#Installing shapely module
!pip install shapely

In [2]:
#Installing folium module
!pip install folium

In [3]:
#Installing pyproj module
!pip install pyproj

In [4]:
#Import libraries
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import math
import shapely.geometry
import pyproj
import warnings
warnings.filterwarnings('ignore')

from pandas.io.json import json_normalize

Then, we will create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 4x4 killometers centered around Kitchener and Waterloo Universities/Colleges.

Let's first find the latitude & longitude of Kitchener and Waterloo's Universities/Colleges, using specific, well known address and Google Maps geocoding API.

In [48]:
api_key =  'SECRET'

In [6]:
#Co-rodinates of University of Waterloo
def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'University of Waterloo 200 University Ave W, Waterloo, ON N2L 3G1'
Waterloo = get_coordinates(api_key, address)
print('Coordinate of {}: {}'.format(address, Waterloo))

Coordinate of University of Waterloo 200 University Ave W, Waterloo, ON N2L 3G1: [43.47129950000001, -80.542715]


In [7]:
#Co-rodinates of Kitchener's Conestoga College
def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address2)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address2 = 'Conestoga College 299 Doon Valley Dr, Kitchener, ON N2G 4M4'
Kitchener = get_coordinates(api_key, address2)
print('Coordinate of {}: {}'.format(address2, Kitchener))

Coordinate of Conestoga College 299 Doon Valley Dr, Kitchener, ON N2G 4M4: [43.3895633, -80.4051748]


Now let's create a grid of area candidates, equaly spaced, centered around the University/College areas. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [8]:
#Waterloo
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Waterloo Center longitude={}, latitude={}'.format(Waterloo[1], Waterloo[0]))
x, y = lonlat_to_xy(Waterloo[1], Waterloo[0])
print('Waterloo Center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Waterloo Center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Waterloo Center longitude=-80.542715, latitude=43.47129950000001
Waterloo Center UTM X=-5322667.908010452, Y=10645104.040561661
Waterloo Center longitude=-80.54271500000054, latitude=43.471299499999866


In [9]:
#Kitchener
def lonlat_to_xy2(lon2, lat2):
    proj_latlon2 = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy2 = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy2 = pyproj.transform(proj_latlon2, proj_xy2, lon2, lat2)
    return xy2[0], xy2[1]

def xy_to_lonlat2(x2, y2):
    proj_latlon2 = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy2 = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat2 = pyproj.transform(proj_xy2, proj_latlon2, x2, y2)
    return lonlat2[0], lonlat2[1]

def calc_xy_distance2(x1, y1, x2, y2):
    dx2 = x2 - x1
    dy2 = y2 - y1
    return math.sqrt(dx2*dx2 + dy2*dy2)

print('Coordinate transformation check')
print('-------------------------------')
print('Kitchener Center longitude={}, latitude={}'.format(Kitchener[1], Kitchener[0]))
x2, y2 = lonlat_to_xy2(Kitchener[1], Kitchener[0])
print('Kitchener Center UTM X={}, Y={}'.format(x2, y2))
lo2, la2 = xy_to_lonlat2(x2, y2)
print('Kitchener Center longitude={}, latitude={}'.format(lo2, la2))

Coordinate transformation check
-------------------------------
Kitchener Center longitude=-80.4051748, latitude=43.3895633
Kitchener Center UTM X=-5337868.099955836, Y=10630979.133620983
Kitchener Center longitude=-80.40517480000055, latitude=43.38956329999985


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [10]:
#Waterloo
waterloo_center_x, waterloo_center_y = lonlat_to_xy(Waterloo[1], Waterloo[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = waterloo_center_x - 6000
x_step = 600
y_min = waterloo_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(waterloo_center_x, waterloo_center_y, x, y)
        if (distance_from_center <= 4000):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

162 candidate neighborhood centers generated.


In [11]:
#Kitchener
kitchener_center_x, kitchener_center_y = lonlat_to_xy2(Kitchener[1], Kitchener[0]) # City center in Cartesian coordinates

k2 = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min2 = kitchener_center_x - 6000
x_step2 = 600
y_min2 = kitchener_center_y - 6000 - (int(21/k2)*k2*600 - 12000)/2
y_step2 = 600 * k2 

latitudes2 = []
longitudes2 = []
distances_from_center2 = []
xs2 = []
ys2 = []
for i2 in range(0, int(21/k2)):
    y2 = y_min2 + i2 * y_step2
    x_offset2 = 300 if i2%2==0 else 0
    for j2 in range(0, 21):
        x2 = x_min2 + j2 * x_step2 + x_offset2
        distance_from_center2 = calc_xy_distance2(kitchener_center_x, kitchener_center_y, x2, y2)
        if (distance_from_center2 <= 4000):
            lon2, lat2 = xy_to_lonlat2(x2, y2)
            latitudes2.append(lat2)
            longitudes2.append(lon2)
            distances_from_center2.append(distance_from_center2)
            xs2.append(x2)
            ys2.append(y2)

print(len(latitudes2), 'candidate neighborhood centers generated.')

162 candidate neighborhood centers generated.


In [12]:
#PLEASE REVIEW IN https://nbviewer.jupyter.org/ As Per Instrucions In The Technical Help Section of Coursea Github will not render folium maps
#https://nbviewer.jupyter.org/github/SarahFewster/Capstone-Project---The-Battle-of-Neighborhoods/blob/main/The%20Battle%20of%20Neighborhoods%20Week%201%20Q2.ipynb
#OR View on IBM Cloud: https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/8a057a9e-5566-40d5-a701-75736285158b/view?access_token=282bb461ef599df872cc5de9d3f9960d14b9905748146a4e729ac498cb1f45a2
map_combined = folium.Map(location=Waterloo and Kitchener, zoom_start=12)
folium.Marker(Waterloo, popup='Waterloo - Univerisy Of Waterloo').add_to(map_combined)
folium.Marker(Kitchener, popup='Kitchener - Conestoga College').add_to(map_combined)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_combined) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_combined)
for lat2, lon2 in zip(latitudes2, longitudes2):
      #folium.CircleMarker([lat, lon], radius=2, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_combined) 
    folium.Circle([lat2, lon2], radius=300, color='red', fill=False).add_to(map_combined)
map_combined

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~4km from it's University/College. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [13]:
#Waterloo
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(api_key, Waterloo[0], Waterloo[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(Waterloo[0], Waterloo[1], addr))

Reverse geocoding check
-----------------------
Address of [43.47129950000001, -80.542715] is: Earth Sciences and Chemistry, University of, 200 University Ave W, Waterloo, ON N2L 3G1, Canada


In [14]:
#Kitchener
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(api_key, Kitchener[0], Kitchener[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(Kitchener[0], Kitchener[1], addr))

Reverse geocoding check
-----------------------
Address of [43.3895633, -80.4051748] is: 299 Doon Valley Dr, Kitchener, ON N2G 4M4, Canada


In [15]:
#Waterloo
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Canada', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')

print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [16]:
#Kitchener
print('Obtaining location addresses: ', end='')
addresses2 = []
for lat2, lon2 in zip(latitudes2, longitudes2):
    address2 = get_address(api_key, lat2, lon2)
    if address is None:
        address2 = 'NO ADDRESS'
    address2 = address2.replace(', Canada', '') # We don't need country part of address
    addresses2.append(address2)
    print(' .', end='')

print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [17]:
#Waterloo
addresses[10:30]

['University Ave. / Conestoga College - Waterloo, Waterloo, ON N2J 2W2',
 '332 Marsland Dr, Waterloo, ON N2J 3Z1',
 '200 Dearborn Blvd, Waterloo, ON N2J 4Y3',
 '130 Mt Hope St, Kitchener, ON N2G 4M6',
 '11 John St W, Waterloo, ON N2L 1A6',
 '43 George St, Waterloo, ON N2J 1K6',
 '7 Peppler St, Waterloo, ON N2J 3C2',
 '82 Peppler St, Waterloo, ON N2J 3C8',
 '61 Marshall St, Waterloo, ON N2J 2T3',
 '65 University Ave E, Waterloo, ON N2J 2V9',
 '315 Waterloo Regional Rd 8, Waterloo, ON N2J 3H8',
 '400 Weber St N, Waterloo, ON N2J 3J3',
 '50 Blue Springs Dr, Waterloo, ON N2J 4M4',
 '362 Tealby Pl, Waterloo, ON N2J 0A1',
 '94 Claremont Ave, Kitchener, ON N2M 2P6',
 '157 Belmont Ave, Waterloo, ON N2L 2B2',
 '40 Fullerton St, Waterloo, ON N2L 6L4',
 '50-75 King St S, Waterloo, ON N2J 1P2',
 '15 Bridgeport Rd W, Waterloo, ON N2L 2X9',
 '151 King St N, Waterloo, ON N2J 2Y1']

In [18]:
#Kitchener
addresses2[10:30]

['365 Linden Dr, Cambridge, ON N3H 5L5',
 'ON-401, Kitchener, ON N2P',
 '4610 King St E, Kitchener, ON N2P 2G6',
 '1182 Langdon Dr, Cambridge, ON N1R 5S3',
 '46 Fallbrook Ln, Cambridge, ON N3H 4R8',
 '24 Fallbrook Ln, Cambridge, ON N3H 4R8',
 '1580 Blair Rd, Ayr, ON N0B 1E0',
 'Fountain St S, Cambridge, ON N3H 1J2',
 '740 Linden Dr, Cambridge, ON N3H 5L5',
 '570 Linden Dr, Cambridge, ON N3H 0C9',
 '45 Greensview Dr, Kitchener, ON N2P 2E6',
 '48 Limerick Dr, Kitchener, ON N2P 2H1',
 '150 Gateway Park Dr, Kitchener, ON N2P 2J4',
 '100 Gateway Park Dr, Kitchener, ON N2P 2J4',
 '1536 1 Township Rd, Cambridge, ON N1R 5S3',
 '81 John Bricker Rd, Cambridge, ON N3H 4R8',
 '75 John Bricker Rd, Cambridge, ON N3H 4R8',
 '12 Ashton St, Cambridge, ON N3H 4R8',
 '1740 Blair Rd, Cambridge, ON N3H 4R8',
 '780 Fountain St S, Cambridge, ON N3H 1J2']

In [19]:
#Waterloo Dataframe
df_locations =pd.DataFrame({'Address': addresses,
                           'Latitude': latitudes,
                           'Longitude': longitudes,
                           'X':xs,
                           'Y':ys,
                           'Distance From Center': distances_from_center})
df_locations.tail(20)

,Address,Latitude,Longitude,X,Y,Distance From Center
142,"1211 Fischer-Hallman Rd N, Waterloo, ON N2T 2P5",43.465333,-80.564034,-5.323268e+06,1.064770e+07,2666.458325
143,"Columbia / Fischer-Hallman, Waterloo, ON N2L 5M1",43.469040,-80.564752,-5.322668e+06,1.064770e+07,2598.076211
144,"262 Residence Rd, Waterloo, ON N2L",43.472746,-80.565470,-5.322068e+06,1.064770e+07,2666.458325
145,"Laurelwood Dr, Waterloo, ON N2L",43.476453,-80.566188,-5.321468e+06,1.064770e+07,2861.817604
146,"581 Bearinger Rd, Waterloo, ON N2L",43.480160,-80.566907,-5.320868e+06,1.064770e+07,3160.696126
147,"359 Bearinger Rd, Waterloo, ON N2L",43.483867,-80.567625,-5.320268e+06,1.064770e+07,3536.947837
148,"390 Lakeview Dr, Waterloo, ON N2L 4Z6",43.487575,-80.568343,-5.319668e+06,1.064770e+07,3968.626967
149,"109 Frederick Banting Pl, Waterloo, ON N2T 1C4",43.455616,-80.566645,-5.324768e+06,1.064822e+07,3758.989226
150,"503 Ennismore Pl, Waterloo, ON N2T 2G3",43.459322,-80.567363,-5.324168e+06,1.064822e+07,3459.768778
151,"516 Queensdale Ct, Waterloo, ON N2T 1P7",43.463028,-80.568082,-5.323568e+06,1.064822e+07,3244.996148


In [20]:
#Kitchener Dataframe
df_locations2 =pd.DataFrame({'Address': addresses2,
                           'Latitude': latitudes2,
                           'Longitude': longitudes2,
                           'X':xs2,
                           'Y':ys2,
                           'Distance From Center': distance_from_center2})
df_locations2.tail(20)

,Address,Latitude,Longitude,X,Y,Distance From Center
142,"213 Doon S Dr, Kitchener, ON N2P 2L8",43.383658,-80.426455,-5.338468e+06,1.063358e+07,8286.73639
143,"40 Mitierra Dr, Kitchener, ON N2P 1C8",43.387359,-80.427155,-5.337868e+06,1.063358e+07,8286.73639
144,"Old Carriage / Homer Watson, Kitchener, ON N2P...",43.391061,-80.427854,-5.337268e+06,1.063358e+07,8286.73639
145,"56 Arrowhead Pl, Kitchener, ON N2P 1C3",43.394763,-80.428553,-5.336668e+06,1.063358e+07,8286.73639
146,"35 Green Valley Dr, Kitchener, ON N2P 2A5",43.398466,-80.429253,-5.336068e+06,1.063358e+07,8286.73639
147,"44 Mill Park Dr, Kitchener, ON N2P 1H4",43.402168,-80.429953,-5.335468e+06,1.063358e+07,8286.73639
148,"200 Wabanaki Dr, Kitchener, ON N2C 2E8",43.405871,-80.430653,-5.334868e+06,1.063358e+07,8286.73639
149,"55 George Ayres Dr, Kitchener, ON N2P 0G5",43.373963,-80.429102,-5.339968e+06,1.063410e+07,8286.73639
150,"568 Topper Woods Crescent, Kitchener, ON N2P 2Y3",43.377664,-80.429801,-5.339368e+06,1.063410e+07,8286.73639
151,"360 Doon S Dr, Kitchener, ON N2P 2T6",43.381365,-80.430501,-5.338768e+06,1.063410e+07,8286.73639


In [21]:
#Merging the Waterloo and Kitchener Dataframes into one Dataframe
df3 = df_locations.append(df_locations2)

In [22]:
#Checking to ensure dataframe merged sucessfully
df3.head(20)

,Address,Latitude,Longitude,X,Y,Distance From Center
0,"165 Allen St E, Waterloo, ON N2J 1J7",43.467034,-80.510437,-5.323868e+06,1.064147e+07,3830.143600
1,"125 Waterloo Regional Rd 9, Waterloo, ON N2J 2K3",43.470741,-80.511148,-5.323268e+06,1.064147e+07,3686.461718
2,"Moses Springer Community Centre, 150 Lincoln R...",43.474448,-80.511859,-5.322668e+06,1.064147e+07,3637.306696
3,"219 Dixie Crescent, Waterloo, ON N2J 3K9",43.478155,-80.512570,-5.322068e+06,1.064147e+07,3686.461718
4,"20 Northcrest Pl, Waterloo, ON N2J 3X4",43.481863,-80.513282,-5.321468e+06,1.064147e+07,3830.143600
5,"55 Union St E, Waterloo, ON N2J 1B9",43.461026,-80.513777,-5.324768e+06,1.064199e+07,3758.989226
6,"124 Allen St E, Waterloo, ON N2J 1J4",43.464733,-80.514488,-5.324168e+06,1.064199e+07,3459.768778
7,"75 Bridgeport Rd E, Waterloo, ON N2J 2K1",43.468439,-80.515200,-5.323568e+06,1.064199e+07,3244.996148
8,"99 Elgin Crescent, Waterloo, ON N2J 2S3",43.472146,-80.515911,-5.322968e+06,1.064199e+07,3132.091953
9,"182b Weber St N, Waterloo, ON N2J 3H4",43.475853,-80.516623,-5.322368e+06,1.064199e+07,3132.091953


In [23]:
#checking to see if the merging of the dataframes worked
print('Number of colums in Dataframe : ', len(df_locations.columns))
print('Number of rows in Dataframe : ', len(df_locations.index))
print('Number of colums in Dataframe : ', len(df3.columns))
print('Number of rows in Dataframe : ', len(df3.index))

Number of colums in Dataframe :  6
Number of rows in Dataframe :  162
Number of colums in Dataframe :  6
Number of rows in Dataframe :  324


### Foursquare
Now that we have our location candidates, we will use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in our list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'sushi restaurant' category, as we need info on sushi restaurants in the neighborhood.

Foursquare credentials are defined in cell bellow.

In [47]:
# Define Foursquare Credentials
CLIENT_ID = 'SECRET' # your Foursquare ID
CLIENT_SECRET = 'SECRET' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SECRET
CLIENT_SECRET:SECRET


In [25]:
#Finding Sushi Restauraunts Near Unitversity of Waterloo In Waterloo
search_query ='sushi'
radius = 5000 #(5km)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Waterloo[0], Waterloo[1], VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ZA31CICFNW5O13A2TSRMR5T3WT14KP21BWFPCSRDQAWRZ4FF&client_secret=T1E3POOBTU4WLPDGXOLLKXPJHRWQG2QAGEVGNL5TVJPOQOT0&ll=43.47129950000001,-80.542715&v=20180604&query=sushi&radius=5000&limit=30'

In [26]:
#Finding Sushi Restauraunts Near Conestoga College In Kitchener
search_query2 ='sushi'
radius2 = 5000 #(5km)

url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Kitchener[0], Kitchener[1], VERSION, search_query2, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=ZA31CICFNW5O13A2TSRMR5T3WT14KP21BWFPCSRDQAWRZ4FF&client_secret=T1E3POOBTU4WLPDGXOLLKXPJHRWQG2QAGEVGNL5TVJPOQOT0&ll=43.3895633,-80.4051748&v=20180604&query=sushi&radius=5000&limit=30'

In [27]:
#Finding All Restauraunts In Waterloo
search_query3 ='Restaurant'
radius3 = 100000 #(100km)
LIMIT3 = 100

url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Waterloo[0], Waterloo[1], VERSION, search_query3, radius, LIMIT3)
url3

'https://api.foursquare.com/v2/venues/search?client_id=ZA31CICFNW5O13A2TSRMR5T3WT14KP21BWFPCSRDQAWRZ4FF&client_secret=T1E3POOBTU4WLPDGXOLLKXPJHRWQG2QAGEVGNL5TVJPOQOT0&ll=43.47129950000001,-80.542715&v=20180604&query=Restaurant&radius=5000&limit=100'

In [28]:
#Finding All Restauraunts In Kitchener
search_query4 ='Restaurant'
radius4 = 100000 #(100km)
LIMIT4 = 100

url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Kitchener[0], Kitchener[1], VERSION, search_query4, radius, LIMIT4)
url4

'https://api.foursquare.com/v2/venues/search?client_id=ZA31CICFNW5O13A2TSRMR5T3WT14KP21BWFPCSRDQAWRZ4FF&client_secret=T1E3POOBTU4WLPDGXOLLKXPJHRWQG2QAGEVGNL5TVJPOQOT0&ll=43.3895633,-80.4051748&v=20180604&query=Restaurant&radius=5000&limit=100'

In [29]:
# Waterloo
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe for sushi restauraunts near University of Waterloo
df4 = json_normalize(venues)
df4.head(10)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,55afc40a498e44ca0990bbe9,168 Sushi Buffet Waterloo,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,D4-5 50 Westmount Rd N,43.461442,-80.538030,"[{'label': 'display', 'lat': 43.46144233288407...",1160,N2L 6N9,CA,Waterloo,ON,Canada,"[D4-5 50 Westmount Rd N, Waterloo ON N2L 6N9, ...",NaN,NaN
1,5186dad4498ef49c8faaa9f4,Itamae Sushi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,235 King St N,43.476728,-80.525298,"[{'label': 'display', 'lat': 43.47672824448561...",1531,N2J 2Y8,CA,Waterloo,ON,Canada,"[235 King St N (at University Ave), Waterloo O...",at University Ave,NaN
2,4cfede82c86e5481ca91c74a,Watami Sushi and Sake Bar,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1612639835,False,15 King St N,43.465538,-80.522496,"[{'label': 'display', 'lat': 43.46553769056703...",1754,N2J 2W6,CA,Waterloo,ON,Canada,"[15 King St N (at Erb St W), Waterloo ON N2J 2...",at Erb St W,NaN
3,4df2068f22718759f8199648,Ah-So Sushi(DC),"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1612639835,False,NaN,43.473107,-80.542896,"[{'label': 'display', 'lat': 43.473107, 'lng':...",201,NaN,CA,Waterloo,ON,Canada,"[Waterloo ON, Canada]",NaN,NaN
4,4cb0fcb3562d224b1f611b88,Sushi 99,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,"36 Northfield Dr E, Unit 12",43.503426,-80.530820,"[{'label': 'display', 'lat': 43.50342570876072...",3703,N2L 6A1,CA,Waterloo,ON,Canada,"[36 Northfield Dr E, Unit 12 (at King St N), W...",at King St N,NaN
5,5814fd2a38fa0816c5b07d09,KEN Sushi House,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,256 Phillip St,43.473483,-80.537197,"[{'label': 'display', 'lat': 43.47348313080127...",507,NaN,CA,Waterloo,ON,Canada,"[256 Phillip St (at University Av W), Waterloo...",at University Av W,NaN
6,4b7f3268f964a520521e30e3,Sushi Stars,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,450 King St. E.,43.446617,-80.481674,"[{'label': 'display', 'lat': 43.44661711801076...",5645,N2L 2L6,CA,Kitchener,ON,Canada,"[450 King St. E. (at Madison Ave. N.), Kitchen...",at Madison Ave. N.,NaN
7,5fc8c33c542ed2502ccdda98,Bento Sushi,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1612639835,False,75 University Avenue West,43.473557,-80.528737,"[{'label': 'display', 'lat': 43.47355729088632...",1156,N2L 3C5,CA,Waterloo,ON,Canada,"[75 University Avenue West, Waterloo ON N2L 3C...",NaN,NaN
8,54fdd4cb498e23c1e6bf74ee,Suki Sushi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,345 Erb st w,43.455146,-80.546067,"[{'label': 'display', 'lat': 43.45514627147399...",1818,N2L 1W4,CA,Waterloo,ON,Canada,"[345 Erb st w, Waterloo ON N2L 1W4, Canada]",NaN,NaN
9,5fb8e4f777761c518f22b7a9,Bento Sushi,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1612639835,False,450 Columbia Street West,43.467967,-80.568014,"[{'label': 'display', 'lat': 43.4679668, 'lng'...",2077,N2T 2W1,CA,Waterloo,ON,Canada,"[450 Columbia Street West, Waterloo ON N2T 2W1...",NaN,603175342


In [30]:
# Kitchener
results2 = requests.get(url2).json()
# assign relevant part of JSON to venues
venues2 = results2['response']['venues']

# tranform venues into a dataframe for sushi restauraunts near Conestoga College
df7 = json_normalize(venues2)
df7.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,5fb3a233a66ece2fc3d58234,Bento Sushi,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1612639835,False,299 Doon Valley Drive,43.389864,-80.404782,"[{'label': 'display', 'lat': 43.3898641, 'lng'...",46,N2G 4M4,CA,Kitchener,ON,Canada,"[299 Doon Valley Drive, Kitchener ON N2G 4M4, ...",597164227,NaN
1,4c252bf7f1272d7ffd5584c5,Ye's Sushi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,70 Sportsworld Dr,43.409135,-80.393169,"[{'label': 'display', 'lat': 43.40913477264124...",2385,NaN,CA,Kitchener,ON,Canada,"[70 Sportsworld Dr (at Gateway Park Dr), Kitch...",NaN,at Gateway Park Dr
2,5fb3a4910c6f00024fd23196,Bento Sushi,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1612639835,False,123 Pioneer Drive,43.393963,-80.432003,"[{'label': 'display', 'lat': 43.39396259999999...",2224,N2P 1K8,CA,Kitchener,ON,Canada,"[123 Pioneer Drive, Kitchener ON N2P 1K8, Canada]",602422090,NaN
3,4f2181d1e4b0b69d789b9a4f,Doon Sushi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,601 Doon Village Road,43.398301,-80.446285,"[{'label': 'display', 'lat': 43.39830101789896...",3464,NaN,CA,Kitchener,ON,Canada,"[601 Doon Village Road (Homer Watson Blvd), Ki...",NaN,Homer Watson Blvd
4,517af2d7e4b0696131ca30ca,Lin Hai Sushi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1612639835,False,1401 River Rd,43.433847,-80.427809,"[{'label': 'display', 'lat': 43.433847, 'lng':...",5258,N2A 3X9,CA,Kitchener,ON,Canada,"[1401 River Rd, Kitchener ON N2A 3X9, Canada]",NaN,NaN


In [31]:
# All Waterloo
results3 = requests.get(url3).json()
# assign relevant part of JSON to venues
venues3 = results3['response']['venues']

# tranform venues into a dataframe for restauraunts near Waterloo University
df5 = json_normalize(venues3)
df5.head(20)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4e2df16952b111a1e374f9a3,Chen's Restaurant 橙子餐廳,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1612639836,False,170 University Ave W,at Phillip St,43.472201,-80.538936,"[{'label': 'display', 'lat': 43.47220145197439...",321,N2L 3E9,CA,Waterloo,ON,Canada,"[170 University Ave W (at Phillip St), Waterlo...",NaN
1,4b631946f964a520d6622ae3,Solé Restaurant and Wine Bar,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",v-1612639836,False,83 Erb St W,Father David Bauer Dr.,43.463539,-80.527127,"[{'label': 'display', 'lat': 43.463539, 'lng':...",1527,N2L 6C2,CA,Waterloo,ON,Canada,"[83 Erb St W (Father David Bauer Dr.), Waterlo...",NaN
2,579e766ccd102f2ed7a0e778,King Tin Restaurant 瓊天海鮮酒家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1612639836,False,318 Spruce Street,NaN,43.479122,-80.526460,"[{'label': 'display', 'lat': 43.479122, 'lng':...",1575,N2L 3M7,CA,Waterloo,ON,Canada,"[318 Spruce Street, Waterloo ON N2L 3M7, Canada]",NaN
3,4b6c71ebf964a520ad3a2ce3,Crystal Palace Restaurant 美味棧粵菜館,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1612639836,False,10 King St S,btw Erb & Willis Way,43.464900,-80.522269,"[{'label': 'display', 'lat': 43.4648999041143,...",1798,N2J 1N8,CA,Waterloo,ON,Canada,"[10 King St S (btw Erb & Willis Way), Waterloo...",NaN
4,4b96bb77f964a520ebe034e3,Jia Jia Lok Chinese Restaurant 家家樂,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1612639836,False,255 King Street N,at University Ave W,43.477514,-80.525377,"[{'label': 'display', 'lat': 43.47751412754712...",1562,N2J 4V2,CA,Waterloo,ON,Canada,"[255 King Street N (at University Ave W), Wate...",NaN
5,4b549552f964a52020c127e3,Symposium Café Restaurant & Lounge,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1612639836,False,4 King St. N,Erb St.,43.465226,-80.522108,"[{'label': 'display', 'lat': 43.46522635075343...",1796,N2J 2W7,CA,Waterloo,ON,Canada,"[4 King St. N (Erb St.), Waterloo ON N2J 2W7, ...",146437950
6,5356d10c11d2bbfbf6721509,Friends' Home Restaurant 友家餐廳,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1612639836,False,402 King St N,Weber,43.484400,-80.526300,"[{'label': 'display', 'lat': 43.4844, 'lng': -...",1971,NaN,CA,Waterloo,ON,Canada,"[402 King St N (Weber), Waterloo ON, Canada]",NaN
7,4b200a97f964a5207b2c24e3,Kismet Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1612639836,False,160 University Ave W,at Phillip St.,43.472591,-80.537194,"[{'label': 'display', 'lat': 43.47259104627461...",468,N2L 3E9,CA,Waterloo,ON,Canada,"[160 University Ave W (at Phillip St.), Waterl...",NaN
8,518fa5aa498e31f5de9f327c,Silo Restaurant,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1612639836,False,115 N Water St,NaN,43.468016,-80.546047,"[{'label': 'display', 'lat': 43.468016, 'lng':...",453,14092,CA,Lewiston,NY,Canada,"[115 N Water St, Lewiston NY 14092, Canada]",NaN
9,5b5a4928a5a315002ca35ec8,Mr. S Yin Restaurant 尹師傅,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1612639836,False,NaN,NaN,43.472360,-80.535995,"[{'label': 'display', 'lat': 43.47236, 'lng': ...",555,N2L 6J3,CA,Waterloo,ON,Canada,"[Waterloo ON N2L 6J3, Canada]",NaN


In [32]:
# All Kitchener
results4 = requests.get(url4).json()
# assign relevant part of JSON to venues
venues4 = results4['response']['venues']

# tranform venues into a dataframe for restauraunts near Conestoga College
df8 = json_normalize(venues4)
df8.head(10)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4bcccd148920b713d9c59edc,Modern India Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1612639836,False,4355 King St E,at Sportsworld Dr,43.406960,-80.396880,"[{'label': 'display', 'lat': 43.40696016679312...",2049,N2P 2E9,CA,Kitchener,ON,Canada,"[4355 King St E (at Sportsworld Dr), Kitchener..."
1,56c2632dcd108bebe43a5dd4,Bombay Bhel Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1612639836,False,4336 King Street East,Hellmann Road,43.407860,-80.396372,"[{'label': 'display', 'lat': 43.40786, 'lng': ...",2157,N2P 3W6,CA,Kitchener,ON,Canada,"[4336 King Street East (Hellmann Road), Kitche..."
2,57c74fea498e24c106a0bc6f,Genesis Spanish Restaurant,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S...",v-1612639836,False,NaN,NaN,43.398587,-80.363134,"[{'label': 'display', 'lat': 43.398587, 'lng':...",3545,NaN,CA,NaN,NaN,Canada,[Canada]
3,4cbb24c19552b60cc838de8b,Cambridge Restaurant,[],v-1612639836,False,752 king street,NaN,43.395376,-80.357562,"[{'label': 'display', 'lat': 43.395376, 'lng':...",3905,NaN,CA,Cambridge,ON,Canada,"[752 king street, Cambridge ON, Canada]"
4,4c4b0122c668e21e9f4274f9,Cambridge Family Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1612639836,False,"752 King st, East",NaN,43.395407,-80.357521,"[{'label': 'display', 'lat': 43.395407, 'lng':...",3909,N3h3n9,CA,Cambridge,ON,Canada,"[752 King st, East, Cambridge ON N3h3n9, Canada]"
5,4b7c7470f964a52088942fe3,Shanghai Restaurant,[],v-1612639836,False,King Street,Chuch,43.395262,-80.357426,"[{'label': 'display', 'lat': 43.395262, 'lng':...",3914,NaN,CA,Cambridge,ON,Canada,"[King Street (Chuch), Cambridge ON, Canada]"
6,5fe03385e80f01698ae8fe9d,"HomeTaste - Indian Restaurant, Catering and Ti...","[{'id': '54135bf5e4b08f3d2429dfdd', 'name': 'N...",v-1612639836,False,1601 River Rd East,NaN,43.428770,-80.422145,"[{'label': 'display', 'lat': 43.42876975908951...",4575,N2A 3Y4,CA,Kitchener,ON,Canada,"[1601 River Rd East, Kitchener ON N2A 3Y4, Can..."
7,4d0f3e5271e8a1cd749679bd,Eagle Restaurant,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",v-1612639836,False,1390 Eagle Street N.,Concession,43.404948,-80.347585,"[{'label': 'display', 'lat': 43.40494822033108...",4962,NaN,CA,Cambridge,ON,Canada,"[1390 Eagle Street N. (Concession), Cambridge ..."
8,5674975f498e98ce88afe43e,126 Chinese Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1612639836,False,2922 King St E,King & Morgan,43.429418,-80.435049,"[{'label': 'display', 'lat': 43.429418, 'lng':...",5051,N2A 1A7,CA,Kitchener,ON,Canada,"[2922 King St E (King & Morgan), Kitchener ON ..."
9,4e6ba31445ddadf2d0fe5de6,Hong Kong Seafood Restaurant,[],v-1612639836,False,700 Strasburg Road,NaN,43.404208,-80.472301,"[{'label': 'display', 'lat': 43.40420847077937...",5668,NaN,CA,Kitchener,ON,Canada,"[700 Strasburg Road, Kitchener ON, Canada]"


In [33]:
#Waterloo 
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df4.columns if col.startswith('location.')] + ['id']
df4_filtered = df4.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df4_filtered['categories'] = df4_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df4_filtered.columns = [column.split('.')[-1] for column in df4_filtered.columns]

df4_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,168 Sushi Buffet Waterloo,Sushi Restaurant,D4-5 50 Westmount Rd N,43.461442,-80.538030,"[{'label': 'display', 'lat': 43.46144233288407...",1160,N2L 6N9,CA,Waterloo,ON,Canada,"[D4-5 50 Westmount Rd N, Waterloo ON N2L 6N9, ...",NaN,55afc40a498e44ca0990bbe9
1,Itamae Sushi,Sushi Restaurant,235 King St N,43.476728,-80.525298,"[{'label': 'display', 'lat': 43.47672824448561...",1531,N2J 2Y8,CA,Waterloo,ON,Canada,"[235 King St N (at University Ave), Waterloo O...",at University Ave,5186dad4498ef49c8faaa9f4
2,Watami Sushi and Sake Bar,Japanese Restaurant,15 King St N,43.465538,-80.522496,"[{'label': 'display', 'lat': 43.46553769056703...",1754,N2J 2W6,CA,Waterloo,ON,Canada,"[15 King St N (at Erb St W), Waterloo ON N2J 2...",at Erb St W,4cfede82c86e5481ca91c74a
3,Ah-So Sushi(DC),Japanese Restaurant,NaN,43.473107,-80.542896,"[{'label': 'display', 'lat': 43.473107, 'lng':...",201,NaN,CA,Waterloo,ON,Canada,"[Waterloo ON, Canada]",NaN,4df2068f22718759f8199648
4,Sushi 99,Sushi Restaurant,"36 Northfield Dr E, Unit 12",43.503426,-80.530820,"[{'label': 'display', 'lat': 43.50342570876072...",3703,N2L 6A1,CA,Waterloo,ON,Canada,"[36 Northfield Dr E, Unit 12 (at King St N), W...",at King St N,4cb0fcb3562d224b1f611b88
5,KEN Sushi House,Sushi Restaurant,256 Phillip St,43.473483,-80.537197,"[{'label': 'display', 'lat': 43.47348313080127...",507,NaN,CA,Waterloo,ON,Canada,"[256 Phillip St (at University Av W), Waterloo...",at University Av W,5814fd2a38fa0816c5b07d09
6,Sushi Stars,Sushi Restaurant,450 King St. E.,43.446617,-80.481674,"[{'label': 'display', 'lat': 43.44661711801076...",5645,N2L 2L6,CA,Kitchener,ON,Canada,"[450 King St. E. (at Madison Ave. N.), Kitchen...",at Madison Ave. N.,4b7f3268f964a520521e30e3
7,Bento Sushi,Restaurant,75 University Avenue West,43.473557,-80.528737,"[{'label': 'display', 'lat': 43.47355729088632...",1156,N2L 3C5,CA,Waterloo,ON,Canada,"[75 University Avenue West, Waterloo ON N2L 3C...",NaN,5fc8c33c542ed2502ccdda98
8,Suki Sushi,Sushi Restaurant,345 Erb st w,43.455146,-80.546067,"[{'label': 'display', 'lat': 43.45514627147399...",1818,N2L 1W4,CA,Waterloo,ON,Canada,"[345 Erb st w, Waterloo ON N2L 1W4, Canada]",NaN,54fdd4cb498e23c1e6bf74ee
9,Bento Sushi,Restaurant,450 Columbia Street West,43.467967,-80.568014,"[{'label': 'display', 'lat': 43.4679668, 'lng'...",2077,N2T 2W1,CA,Waterloo,ON,Canada,"[450 Columbia Street West, Waterloo ON N2T 2W1...",NaN,5fb8e4f777761c518f22b7a9


In [34]:
#Kitchener
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in df7.columns if col.startswith('location.')] + ['id']
df7_filtered = df7.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type2(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df7_filtered['categories'] = df7_filtered.apply(get_category_type2, axis=1)

# clean column names by keeping only last term
df7_filtered.columns = [column.split('.')[-1] for column in df7_filtered.columns]

df7_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Bento Sushi,Restaurant,299 Doon Valley Drive,43.389864,-80.404782,"[{'label': 'display', 'lat': 43.3898641, 'lng'...",46,N2G 4M4,CA,Kitchener,ON,Canada,"[299 Doon Valley Drive, Kitchener ON N2G 4M4, ...",NaN,5fb3a233a66ece2fc3d58234
1,Ye's Sushi,Sushi Restaurant,70 Sportsworld Dr,43.409135,-80.393169,"[{'label': 'display', 'lat': 43.40913477264124...",2385,NaN,CA,Kitchener,ON,Canada,"[70 Sportsworld Dr (at Gateway Park Dr), Kitch...",at Gateway Park Dr,4c252bf7f1272d7ffd5584c5
2,Bento Sushi,Restaurant,123 Pioneer Drive,43.393963,-80.432003,"[{'label': 'display', 'lat': 43.39396259999999...",2224,N2P 1K8,CA,Kitchener,ON,Canada,"[123 Pioneer Drive, Kitchener ON N2P 1K8, Canada]",NaN,5fb3a4910c6f00024fd23196
3,Doon Sushi,Sushi Restaurant,601 Doon Village Road,43.398301,-80.446285,"[{'label': 'display', 'lat': 43.39830101789896...",3464,NaN,CA,Kitchener,ON,Canada,"[601 Doon Village Road (Homer Watson Blvd), Ki...",Homer Watson Blvd,4f2181d1e4b0b69d789b9a4f
4,Lin Hai Sushi,Sushi Restaurant,1401 River Rd,43.433847,-80.427809,"[{'label': 'display', 'lat': 43.433847, 'lng':...",5258,N2A 3X9,CA,Kitchener,ON,Canada,"[1401 River Rd, Kitchener ON N2A 3X9, Canada]",NaN,517af2d7e4b0696131ca30ca


In [35]:
#All Near Waterloo
# keep only columns that include venue name, and anything that is associated with location
filtered_columns3 = ['name', 'categories'] + [col for col in df5.columns if col.startswith('location.')] + ['id']
df5_filtered = df5.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type3(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df5_filtered['categories'] = df5_filtered.apply(get_category_type3, axis=1)

# clean column names by keeping only last term
df5_filtered.columns = [column.split('.')[-1] for column in df5_filtered.columns]

df5_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Chen's Restaurant 橙子餐廳,Chinese Restaurant,170 University Ave W,43.472201,-80.538936,"[{'label': 'display', 'lat': 43.47220145197439...",321,N2L 3E9,CA,Waterloo,ON,Canada,"[170 University Ave W (at Phillip St), Waterlo...",at Phillip St,4e2df16952b111a1e374f9a3
1,Solé Restaurant and Wine Bar,Wine Bar,83 Erb St W,43.463539,-80.527127,"[{'label': 'display', 'lat': 43.463539, 'lng':...",1527,N2L 6C2,CA,Waterloo,ON,Canada,"[83 Erb St W (Father David Bauer Dr.), Waterlo...",Father David Bauer Dr.,4b631946f964a520d6622ae3
2,King Tin Restaurant 瓊天海鮮酒家,Chinese Restaurant,318 Spruce Street,43.479122,-80.526460,"[{'label': 'display', 'lat': 43.479122, 'lng':...",1575,N2L 3M7,CA,Waterloo,ON,Canada,"[318 Spruce Street, Waterloo ON N2L 3M7, Canada]",NaN,579e766ccd102f2ed7a0e778
3,Crystal Palace Restaurant 美味棧粵菜館,Dim Sum Restaurant,10 King St S,43.464900,-80.522269,"[{'label': 'display', 'lat': 43.4648999041143,...",1798,N2J 1N8,CA,Waterloo,ON,Canada,"[10 King St S (btw Erb & Willis Way), Waterloo...",btw Erb & Willis Way,4b6c71ebf964a520ad3a2ce3
4,Jia Jia Lok Chinese Restaurant 家家樂,Chinese Restaurant,255 King Street N,43.477514,-80.525377,"[{'label': 'display', 'lat': 43.47751412754712...",1562,N2J 4V2,CA,Waterloo,ON,Canada,"[255 King Street N (at University Ave W), Wate...",at University Ave W,4b96bb77f964a520ebe034e3
5,Symposium Café Restaurant & Lounge,Restaurant,4 King St. N,43.465226,-80.522108,"[{'label': 'display', 'lat': 43.46522635075343...",1796,N2J 2W7,CA,Waterloo,ON,Canada,"[4 King St. N (Erb St.), Waterloo ON N2J 2W7, ...",Erb St.,4b549552f964a52020c127e3
6,Friends' Home Restaurant 友家餐廳,Chinese Restaurant,402 King St N,43.484400,-80.526300,"[{'label': 'display', 'lat': 43.4844, 'lng': -...",1971,NaN,CA,Waterloo,ON,Canada,"[402 King St N (Weber), Waterloo ON, Canada]",Weber,5356d10c11d2bbfbf6721509
7,Kismet Restaurant,Indian Restaurant,160 University Ave W,43.472591,-80.537194,"[{'label': 'display', 'lat': 43.47259104627461...",468,N2L 3E9,CA,Waterloo,ON,Canada,"[160 University Ave W (at Phillip St.), Waterl...",at Phillip St.,4b200a97f964a5207b2c24e3
8,Silo Restaurant,American Restaurant,115 N Water St,43.468016,-80.546047,"[{'label': 'display', 'lat': 43.468016, 'lng':...",453,14092,CA,Lewiston,NY,Canada,"[115 N Water St, Lewiston NY 14092, Canada]",NaN,518fa5aa498e31f5de9f327c
9,Mr. S Yin Restaurant 尹師傅,Chinese Restaurant,NaN,43.472360,-80.535995,"[{'label': 'display', 'lat': 43.47236, 'lng': ...",555,N2L 6J3,CA,Waterloo,ON,Canada,"[Waterloo ON N2L 6J3, Canada]",NaN,5b5a4928a5a315002ca35ec8


In [36]:
#All Near Kitchener
# keep only columns that include venue name, and anything that is associated with location
filtered_columns4 = ['name', 'categories'] + [col for col in df8.columns if col.startswith('location.')] + ['id']
df8_filtered = df8.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type4(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df8_filtered['categories'] = df8_filtered.apply(get_category_type4, axis=1)

# clean column names by keeping only last term
df8_filtered.columns = [column.split('.')[-1] for column in df8_filtered.columns]

df8_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Modern India Restaurant,Indian Restaurant,4355 King St E,43.406960,-80.396880,"[{'label': 'display', 'lat': 43.40696016679312...",2049,N2P 2E9,CA,Kitchener,ON,Canada,"[4355 King St E (at Sportsworld Dr), Kitchener...",at Sportsworld Dr,4bcccd148920b713d9c59edc
1,Bombay Bhel Restaurant,Indian Restaurant,4336 King Street East,43.407860,-80.396372,"[{'label': 'display', 'lat': 43.40786, 'lng': ...",2157,N2P 3W6,CA,Kitchener,ON,Canada,"[4336 King Street East (Hellmann Road), Kitche...",Hellmann Road,56c2632dcd108bebe43a5dd4
2,Genesis Spanish Restaurant,Spanish Restaurant,NaN,43.398587,-80.363134,"[{'label': 'display', 'lat': 43.398587, 'lng':...",3545,NaN,CA,NaN,NaN,Canada,[Canada],NaN,57c74fea498e24c106a0bc6f
3,Cambridge Restaurant,None,752 king street,43.395376,-80.357562,"[{'label': 'display', 'lat': 43.395376, 'lng':...",3905,NaN,CA,Cambridge,ON,Canada,"[752 king street, Cambridge ON, Canada]",NaN,4cbb24c19552b60cc838de8b
4,Cambridge Family Restaurant,Restaurant,"752 King st, East",43.395407,-80.357521,"[{'label': 'display', 'lat': 43.395407, 'lng':...",3909,N3h3n9,CA,Cambridge,ON,Canada,"[752 King st, East, Cambridge ON N3h3n9, Canada]",NaN,4c4b0122c668e21e9f4274f9
5,Shanghai Restaurant,None,King Street,43.395262,-80.357426,"[{'label': 'display', 'lat': 43.395262, 'lng':...",3914,NaN,CA,Cambridge,ON,Canada,"[King Street (Chuch), Cambridge ON, Canada]",Chuch,4b7c7470f964a52088942fe3
6,"HomeTaste - Indian Restaurant, Catering and Ti...",North Indian Restaurant,1601 River Rd East,43.428770,-80.422145,"[{'label': 'display', 'lat': 43.42876975908951...",4575,N2A 3Y4,CA,Kitchener,ON,Canada,"[1601 River Rd East, Kitchener ON N2A 3Y4, Can...",NaN,5fe03385e80f01698ae8fe9d
7,Eagle Restaurant,Food & Drink Shop,1390 Eagle Street N.,43.404948,-80.347585,"[{'label': 'display', 'lat': 43.40494822033108...",4962,NaN,CA,Cambridge,ON,Canada,"[1390 Eagle Street N. (Concession), Cambridge ...",Concession,4d0f3e5271e8a1cd749679bd
8,126 Chinese Restaurant,Restaurant,2922 King St E,43.429418,-80.435049,"[{'label': 'display', 'lat': 43.429418, 'lng':...",5051,N2A 1A7,CA,Kitchener,ON,Canada,"[2922 King St E (King & Morgan), Kitchener ON ...",King & Morgan,5674975f498e98ce88afe43e
9,Hong Kong Seafood Restaurant,None,700 Strasburg Road,43.404208,-80.472301,"[{'label': 'display', 'lat': 43.40420847077937...",5668,NaN,CA,Kitchener,ON,Canada,"[700 Strasburg Road, Kitchener ON, Canada]",NaN,4e6ba31445ddadf2d0fe5de6


In [37]:
#Waterloo Determining Average number or restauraunts in neighbourhood
Number_of_Waterloo_Sushi_Venues = len(df4.index)
Number_of_Waterloo_All_Venues = len(df5.index)

def averagewaerloo (Number_of_Waterloo_Sushi_Venues,Number_of_Waterloo_All_Venues):
    return (Number_of_Waterloo_Sushi_Venues + Number_of_Waterloo_All_Venues)/2
avgw = averagewaerloo(int(Number_of_Waterloo_Sushi_Venues),int(Number_of_Waterloo_All_Venues))


#Waterloo
print('Total number of restaurants Waterloo:', len(df5))
print('Total number of Waterloo sushi restaurants:', len(df4))
print('Percentage of sushi restaurants Waterloo: {:.2f}%'.format(len(df4) / len(df5) * 100))
print('Average number of restaurants in Waterloo neighborhood', avgw)


Total number of restaurants Waterloo: 50
Total number of Waterloo sushi restaurants: 24
Percentage of sushi restaurants Waterloo: 48.00%
Average number of restaurants in Waterloo neighborhood 37.0


In [38]:
#Kitchener Determining Average number or restauraunts in neighbourhood
Number_of_Kitchener_Sushi_Venues = len(df7.index)
Number_of_Kitchener_All_Venues = len(df8.index)

def averagekitchener (Number_of_Kitchener_Sushi_Venues,Number_of_Kitchener_All_Venues):
    return (Number_of_Kitchener_Sushi_Venues + Number_of_Kitchener_All_Venues)/2
avgk = averagekitchener(int(Number_of_Kitchener_Sushi_Venues),int(Number_of_Kitchener_All_Venues))


#Kitcener
print('Total number of restaurants Kitchener:', len(df8))
print('Total number of Kitchener sushi restaurants:', len(df7))
print('Percentage of sushi restaurants Kitchener: {:.2f}%'.format(len(df7) / len(df8) * 100))
print('Average number of restaurants in Kitchener neighborhood', avgk)


Total number of restaurants Kitchener: 14
Total number of Kitchener sushi restaurants: 5
Percentage of sushi restaurants Kitchener: 35.71%
Average number of restaurants in Kitchener neighborhood 9.5


We'll view all the collected restaurants in our area of interest on a map, and also show sushi restaurants in different color.

In [39]:
#PLEASE REVIEW IN https://nbviewer.jupyter.org/ As Per Instrucions In The Technical Help Section of Coursea Github will not render folium maps
#https://nbviewer.jupyter.org/github/SarahFewster/Capstone-Project---The-Battle-of-Neighborhoods/blob/main/The%20Battle%20of%20Neighborhoods%20Week%201%20Q2.ipynb
#OR View on IBM Cloud: https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/8a057a9e-5566-40d5-a701-75736285158b/view?access_token=282bb461ef599df872cc5de9d3f9960d14b9905748146a4e729ac498cb1f45a2
#Combining Information on Shushi Restauraunts in the areas of Kitchener and Waterloo 
kw_map = folium.Map(location=[lat,lon], zoom_start=12) # generate map centred around Kitchener/ Waterloo area

folium.Marker(Waterloo, popup='Waterloo - University of Waterloo').add_to(kw_map)
folium.Marker(Kitchener, popup='Kitchener - Conestoga College').add_to(kw_map)

# add the Waterloo sushi restaurants as Purple and Blue circle markers
for lat, lng, label in zip(df4_filtered.lat, df4_filtered.lng, df4_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='Purple',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(kw_map)

# add the Kitchener sushi restaurants as Red and Blue circle markers
for lat, lng, label in zip(df7_filtered.lat, df7_filtered.lng, df7_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(kw_map)
    
# add all the Waterloo restaurants as Orange and Black circle markers
for lat, lng, label in zip(df5_filtered.lat, df5_filtered.lng, df5_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(kw_map)
    
# add the Kitchener restaurants as Green and Black circle markers
for lat, lng, label in zip(df8_filtered.lat, df8_filtered.lng, df8_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(kw_map)

# display map
kw_map

Now we have all the restaurants in area within few kilometers our respecive Universities/Colleges, and we know which ones are sushi restaurants! We also know which restaurants are in vicinity of Kitchener/ Waterloo area.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new sushi restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Kitchener/ Waterloo that have low restaurant density, particularly those with low number of sushi restaurants. We limit our analysis to an area ~5km around Universities and Colleges.

In first step we have collected the required **data: location and type (category) of every restaurant within 5km from Kitchener/Waterloo Universities and Colleges**. We have also **identified sushi restaurants**.

In the final step we will explore our original question, which is the best neighbourhood to start a new sushi restauraunt Kitchener or Waterloo or neither?

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data colectively. Let's find the average number of sushi restauraunts in these two areas combined. 

In [40]:
#merging list of all restauraunts to one dataframe for the Kitchener/ Waterloo Area
location_restaurants = df8_filtered.append(df5_filtered)
location_restaurants.reset_index(inplace=True)

In [41]:
#checking to see if the merging of the dataframes worked
print('Number of colums in Dataframe : ', len(df8_filtered.columns))
print('Number of rows in Dataframe : ', len(df8_filtered.index))
print('Number of colums in Dataframe : ', len(df5_filtered.columns))
print('Number of rows in Dataframe : ', len(df5_filtered.index))
print('Number of colums in Dataframe : ', len(location_restaurants.columns))
print('Number of rows in Dataframe : ', len(location_restaurants.index))

Number of colums in Dataframe :  15
Number of rows in Dataframe :  14
Number of colums in Dataframe :  15
Number of rows in Dataframe :  50
Number of colums in Dataframe :  16
Number of rows in Dataframe :  64


In [42]:
#viewing new dataframe
location_restaurants.head()

,index,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,0,Modern India Restaurant,Indian Restaurant,4355 King St E,43.406960,-80.396880,"[{'label': 'display', 'lat': 43.40696016679312...",2049,N2P 2E9,CA,Kitchener,ON,Canada,"[4355 King St E (at Sportsworld Dr), Kitchener...",at Sportsworld Dr,4bcccd148920b713d9c59edc
1,1,Bombay Bhel Restaurant,Indian Restaurant,4336 King Street East,43.407860,-80.396372,"[{'label': 'display', 'lat': 43.40786, 'lng': ...",2157,N2P 3W6,CA,Kitchener,ON,Canada,"[4336 King Street East (Hellmann Road), Kitche...",Hellmann Road,56c2632dcd108bebe43a5dd4
2,2,Genesis Spanish Restaurant,Spanish Restaurant,NaN,43.398587,-80.363134,"[{'label': 'display', 'lat': 43.398587, 'lng':...",3545,NaN,CA,NaN,NaN,Canada,[Canada],NaN,57c74fea498e24c106a0bc6f
3,3,Cambridge Restaurant,None,752 king street,43.395376,-80.357562,"[{'label': 'display', 'lat': 43.395376, 'lng':...",3905,NaN,CA,Cambridge,ON,Canada,"[752 king street, Cambridge ON, Canada]",NaN,4cbb24c19552b60cc838de8b
4,4,Cambridge Family Restaurant,Restaurant,"752 King st, East",43.395407,-80.357521,"[{'label': 'display', 'lat': 43.395407, 'lng':...",3909,N3h3n9,CA,Cambridge,ON,Canada,"[752 King st, East, Cambridge ON N3h3n9, Canada]",NaN,4c4b0122c668e21e9f4274f9


In [43]:
#merging list of all restauraunts to one dataframe for the Kitchener/ Waterloo Area
location_sushi_restaurants = df7_filtered.append(df4_filtered)
location_sushi_restaurants.reset_index(inplace=True)

In [44]:
#checking to see if the merging of the dataframes worked
print('Number of colums in Dataframe : ', len(df4_filtered.columns))
print('Number of rows in Dataframe : ', len(df4_filtered.index))
print('Number of colums in Dataframe : ', len(df7_filtered.columns))
print('Number of rows in Dataframe : ', len(df7_filtered.index))
print('Number of colums in Dataframe : ', len(location_sushi_restaurants.columns))
print('Number of rows in Dataframe : ', len(location_sushi_restaurants.index))

Number of colums in Dataframe :  15
Number of rows in Dataframe :  24
Number of colums in Dataframe :  15
Number of rows in Dataframe :  5
Number of colums in Dataframe :  16
Number of rows in Dataframe :  29


In [45]:
#viewing new dataframe
location_sushi_restaurants.head()

,index,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,0,Bento Sushi,Restaurant,299 Doon Valley Drive,43.389864,-80.404782,"[{'label': 'display', 'lat': 43.3898641, 'lng'...",46,N2G 4M4,CA,Kitchener,ON,Canada,"[299 Doon Valley Drive, Kitchener ON N2G 4M4, ...",NaN,5fb3a233a66ece2fc3d58234
1,1,Ye's Sushi,Sushi Restaurant,70 Sportsworld Dr,43.409135,-80.393169,"[{'label': 'display', 'lat': 43.40913477264124...",2385,NaN,CA,Kitchener,ON,Canada,"[70 Sportsworld Dr (at Gateway Park Dr), Kitch...",at Gateway Park Dr,4c252bf7f1272d7ffd5584c5
2,2,Bento Sushi,Restaurant,123 Pioneer Drive,43.393963,-80.432003,"[{'label': 'display', 'lat': 43.39396259999999...",2224,N2P 1K8,CA,Kitchener,ON,Canada,"[123 Pioneer Drive, Kitchener ON N2P 1K8, Canada]",NaN,5fb3a4910c6f00024fd23196
3,3,Doon Sushi,Sushi Restaurant,601 Doon Village Road,43.398301,-80.446285,"[{'label': 'display', 'lat': 43.39830101789896...",3464,NaN,CA,Kitchener,ON,Canada,"[601 Doon Village Road (Homer Watson Blvd), Ki...",Homer Watson Blvd,4f2181d1e4b0b69d789b9a4f
4,4,Lin Hai Sushi,Sushi Restaurant,1401 River Rd,43.433847,-80.427809,"[{'label': 'display', 'lat': 43.433847, 'lng':...",5258,N2A 3X9,CA,Kitchener,ON,Canada,"[1401 River Rd, Kitchener ON N2A 3X9, Canada]",NaN,517af2d7e4b0696131ca30ca


In [46]:
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())
print('Average number of sushi restaurants in neighborhood:', np.array([len(r) for r in location_sushi_restaurants]).mean())

Average number of restaurants in neighborhood: 6.9375
Average number of sushi restaurants in neighborhood: 6.9375


## Results and Discussion <a name="results"></a>

Our analysis shows that there is a great number of restaurants in Waterloo, there are few pockets of low restaurant density fairly close to the University areas. The Highest concentration of restaurants was detected near the University and uptown Waterloo. Other areas of Waterloo away from the Universities may be able to support a new type of restauraunt but we were focused on the areas neare the Universities wich offers a high traffic area and a certain popularity among students.

Our analysis also shows that although there is a good number of restaurants in Kitchener, there are pockets of low restaurant density fairly close to Conestoga College's Doon campus. The highest concentration of restaurants was detected near Kitchener's city centerso our focus and interest in the area around the area of the college when looking at all restauraunts seems to suggest it could support a new restauraunt which would still offer a popularity among students, and a high traffic area.

After defining our neighbourhoods of Kitchener and Waterloo and looking at all restauraunts in the area we then directed our attention to sushi restauraunts in the area covering an approximately 5 KM area from each location. 

Location candidates were geocoded to be used as markers and positioned on a map for a more detailed view of the restauraunts and sushi restauraunts in the arrea. 

Result of all this is a list of 64 restauraunts cumulatively in Kitchener/ Waterloo as well as 29 sushi restauraunts cumulatively in Kitchener/ Waterloo. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Kitchener/ Waterloo Universities and Colleges but not crowded with existing restaurants (particularly sushi) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify if the Kitchener or Waterloo area could sustain another sushi restauraunt near the College/ Univerisity areas. We were looking for a low number of restaurants (particularly sushi restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new sushi restaurant. By maping restaurant density distribution from Foursquare data we have visually shown areas which have a high density of restauraunts, a low density of restauraunts, a high density of sushi restauraunts, and low densities of sushi restauraunts. 

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.

However, i think our initial analysys of these two areas shows them to be rather full of sushi restauraunts in our areas of interest and these stakeholders might have a better time if they expanded their search to include a Cambridge or Elmira location rather than the Kitchener/ Waterloo area's which already seem to have a rather competitive market around the University/ College area. 